In [1]:
import mlflow
import logging
import time
import pandas as pd
import mlflow.sklearn as ms
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import numpy as np

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
train = pd.read_csv('drugsComTrain_raw.csv', nrows=3000).reset_index(drop=True)
test = pd.read_csv('drugsComTest_raw.csv', nrows=2000).reset_index(drop=True)

df = pd.concat([train, test]).reset_index(drop=True)
df.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [3]:
df.rating.unique()

array([ 9,  8,  5,  2,  1, 10,  4,  3,  7,  6], dtype=int64)

In [4]:
df['rating'] = np.where(df['rating'] <= 5, 'negative', 'positive')
df.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",positive,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",positive,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",negative,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",positive,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",positive,27-Nov-16,37


In [5]:
df.tail()

,uniqueID,drugName,condition,review,rating,date,usefulCount
4995,98129,Nexplanon,Birth Control,"""I love it! I&#039;m too forgetful for the pil...",positive,27-Jul-15,2
4996,163590,Mirtazapine,Depression,"""I have several issues and was given Remeron a...",positive,8-Aug-14,108
4997,52007,Adipex-P,Weight Loss,"""So I have always been a big girl! I have 3 ki...",positive,26-Mar-15,92
4998,3806,Fioricet,Headache,"""I tried this medicine today with no headache ...",negative,13-May-10,13
4999,44573,Asacol,Ulcerative Colitis,"""I was diagnosed with ulcerative colitis in Ap...",positive,10-Feb-12,33


In [6]:
df.to_csv('data.csv', index=False)

In [7]:
df = pd.read_csv('data.csv')
df.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",positive,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",positive,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",negative,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",positive,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",positive,27-Nov-16,37


In [8]:
df.rating.value_counts(normalize=True).round(3)

rating
positive    0.701
negative    0.299
Name: proportion, dtype: float64

In [9]:
df = df[['review', 'rating']]
df.head()

,review,rating
0,"""It has no side effect, I take it in combinati...",positive
1,"""My son is halfway through his fourth week of ...",positive
2,"""I used to take another oral contraceptive, wh...",negative
3,"""This is my first time using any form of birth...",positive
4,"""Suboxone has completely turned my life around...",positive


In [10]:
def transform_text(text):
    "Transform the text by text lowering, tokenizing, removing special characters, stopwords, punctuations and stemming."
    
    # Lower case the text
    text = text.lower()

    # Tokenize the text
    text = nltk.word_tokenize(text)

    # Remove special characters
    text = [word for word in text if word.isalnum()]

    # Remove stopwords and punctuations
    text = [word for word in text if word not in stopwords.words('english') and word not in string.punctuation]

    # Stemming the texts
    ps = PorterStemmer()
    text = [ps.stem(word) for word in text]

    # Join the tokens back into a single string
    return ' '.join(text)

df.loc[:, 'review'] = df['review'].apply(transform_text)
df.head()

,review,rating
0,side effect take combin bystol 5 mg fish oil,positive
1,son halfway fourth week intuniv becam concern ...,positive
2,use take anoth oral contracept 21 pill cycl li...,negative
3,first time use form birth control 039 glad wen...,positive
4,suboxon complet turn life around feel healthie...,positive


In [11]:
df['rating'] = df['rating'].map({'positive': 1, 'negative': 0})
df.head()

,review,rating
0,side effect take combin bystol 5 mg fish oil,1
1,son halfway fourth week intuniv becam concern ...,1
2,use take anoth oral contracept 21 pill cycl li...,0
3,first time use form birth control 039 glad wen...,1
4,suboxon complet turn life around feel healthie...,1


In [12]:
df.isnull().sum()

review    0
rating    0
dtype: int64

In [13]:
df.duplicated().sum()

59

In [14]:
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df.tail()

,review,rating
4936,love 039 forget pill actual togeth stop get pe...,1
4937,sever issu given remeron along seroquel tegret...,1
4938,alway big girl 3 kid pregnanc ad weight carri ...,1
4939,tri medicin today headach relief get drowsi mu...,0
4940,diagnos ulcer coliti april numer visit doctor ...,1


In [15]:
df.rating.value_counts()

rating
1    3464
0    1477
Name: count, dtype: int64

In [16]:
NUM_FEATURES = 100
vectorizer = CountVectorizer(max_features=NUM_FEATURES)
X = vectorizer.fit_transform(df['review'])
y = df['rating']

In [17]:
type(X)

scipy.sparse._csr.csr_matrix

In [18]:
TEST_SIZE = 0.25
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=42)

In [19]:
import dagshub
mlflow.set_tracking_uri('https://dagshub.com/nafiul-araf/Drug-Review-Project-MLFlow-.mlflow')

dagshub.init(repo_owner='nafiul-araf', repo_name='Drug-Review-Project-MLFlow-', mlflow=True)

mlflow.set_experiment('Logistic Regression Baseline')

Accessing as nafiul-araf

Initialized MLflow to track repo "nafiul-araf/Drug-Review-Project-MLFlow-"

Repository nafiul-araf/Drug-Review-Project-MLFlow- initialized!

<Experiment: artifact_location='mlflow-artifacts:/938f099f6a744bd3a095ed43f2f82b46', creation_time=1746867082134, experiment_id='0', last_update_time=1746867082134, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [20]:
# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info('Starting MLFlow run.........')

with mlflow.start_run():
    start_time = time.time()

    try:
        logging.info('Logging the parameters......')
        mlflow.log_params({
            'vectorizer': 'Bag of Words',
            'num_features': NUM_FEATURES,
            'test_size': TEST_SIZE
        })

        logging.info('Initializing Logistic Regression Model.......')
        model_lr = LogisticRegression(max_iter=1000)

        logging.info('Fitting the model on the training data.......')
        model_lr.fit(X_train, y_train)
        mlflow.log_metric('training accuarcy', model_lr.score(X_train, y_train))
        
        logging.info('Starting the prediction........')
        y_pred = model_lr.predict(X_test)

        logging.info('Logging the evaluation metrics.........')
        mlflow.log_metrics({
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1': f1_score(y_test, y_pred)
        })


        logging.info('Saving and logging the model')
        ms.log_model(model_lr, 'baseline_logistic_regression_model')


        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")


        logging.info(f"Accuracy: {accuracy_score(y_test, y_pred)}")
        logging.info(f"Precision: {precision_score(y_test, y_pred)}")
        logging.info(f"Recall: {recall_score(y_test, y_pred)}")
        logging.info(f"F1 Score: {f1_score(y_test, y_pred)}")
    
    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-05-10 15:17:04,675 - INFO - Starting MLFlow run.........
2025-05-10 15:17:05,618 - INFO - Logging the parameters......
2025-05-10 15:17:06,133 - INFO - Initializing Logistic Regression Model.......
2025-05-10 15:17:06,138 - INFO - Fitting the model on the training data.......
2025-05-10 15:17:06,666 - INFO - Starting the prediction........
2025-05-10 15:17:06,672 - INFO - Logging the evaluation metrics.........
2025-05-10 15:17:07,115 - INFO - Saving and logging the model
2025/05/10 15:17:30 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025-05-10 15:17:30,425 - INFO - Model training and logging completed in 24.81 seconds.
2025-05-10 15:17:30,437 - INFO - Accuracy: 0.7411003236245954
2025-05-10 15:17:30,483 - INFO - Precision: 0.7564102564102564
2025-05-10 15:17:30,544 - INFO - Recall: 0.9386363636363636
2025-05-10 15:17:30,560 - INFO - F1 Score: 0.8377281947261663
2025/05/10 15